In [167]:
from __future__ import print_function  # used for making Python 2 print function act like python3 function

In [1]:
import pandas as pd
%matplotlib inline

# Stroop Data Analysis in Pandas

## 1. Load Data into a Pandas DataFrame

In [2]:
df = pd.read_csv('StroopData.csv')
df.head()

,respcolor,stimcolor,trialnum,subject,stimword,reaction_time
0,Y,Y,1.0,156772.0,Y,1389.0
1,G,G,2.0,156772.0,G,2043.0
2,R,R,3.0,156772.0,R,1389.0
3,Y,Y,4.0,156772.0,Y,1092.0
4,G,G,5.0,156772.0,G,1310.0


How Many Trials were there in total, across all subjects?

What was the Maximum number of trials any subject did? The minimum?

How Many subjects were there in this study?

What is the mean response Time, across all subjects and conditions?

Plot a histogram of the response times

What was the mean response time for each subject?

Log transform the reaction Times

Reject Bad Data: Log Reaction Time > 5.7

From here on, use only the accepted data!

## Overall, what was the accuracy of the responses?

  - How many trials, in total, were correct (stimcolor matched respcolor)?
  - How many trials were incorrect?
  - What percent accuract was there, overall?

## Matched vs Unmatched Colors

 - What weas the mean log response time for Congruent trials (the stimulus word and color matched each other)?
 - What was the mean log response time for Incongruent trials (when the stimulus word and color did not match each other)?

## Make a Bar Graph of Mean Log Response Time

## Make a Scatter Plot: Change in Performance over Time

## Do all subjects show a difference in response time between the Congruent and Incogruent conditions, or is this just coming from a few subjects?

Plot a histogram of the difference in mean log reaction times between conditions for each subject.  What do you think, should anyone be rejected?

## Plot the mean reaction times for each condition, for each subject.

## Participant Response Speed vs Size of Stroop Effect
Make a scatter plot showing the relationship between how quickly subjects responded for congruent and incongruent stimuli (mean response to each stimulus category)!